In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")
open_ai_model = "gpt-5-nano-2025-08-07"

Enter Your OpenAI API Key:··········


# **ReAct Prompting Framework Overview**

The ReAct framework was [introduced in a 2022 paper, ReAct: Synergizing Reasoning and Acting in Language Models, by researchers at Google](https://blog.research.google/2022/11/react-synergizing-reasoning-and-acting.html).

### 🌐 **ReAct (Reasoning + Acting) Explained**

- Blends reasoning with action in language models.

- Generates reasoning traces alongside task-specific actions.

### 🔗 **Capabilities of ReAct**

1. 🔄 **Interleaved Reasoning and Acting**: Fuses reasoning with actions for fluid updates.

2. 🌍 **External Interactions**: Accesses external sources like Wikipedia for more info.

3. 🧑‍🎨 **Human-like Problem Solving**: Offers easy-to-follow, logical trajectories.

4. 🛠️ **Wide-Ranging Use Cases**: Ideal for varied tasks like QA, text games, web browsing.

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2210.03629/assets/x1.png" alt="Image Description" style="width:100%">
  <figcaption>Comparison of four prompting methods, (a) Standard, (b) Chain-of-thought (CoT, Reason Only), (c) Act-only, and (d) (Reason+Act), solving a HotpotQA question. (2) Comparison of (a) Act-only and (b) prompting methods to solve an interactive AlfWorld task. In both domains, the initial prompts of in-context examples are ommitted, and only show task solving trajectories generated by the model (Act, Thought) and the environment (Obs).</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2210.03629">Image Source</a>
</figure>


### 🌟 **Why ReAct Stands Out**

1. **Balanced Emphasis on Reasoning & Acting**: Surpasses models limited to just reasoning or action.

2. **Engages with External Data**: Pulls in real-time info for better accuracy.

3. **Boosted Transparency**: Step-by-step approach clarifies decision-making.


# 🤖 **Overview of ReAct in LangChain**

- ReAct is an agent type in LangChain.

- It uses the ReAct framework to pick tools based on their descriptions.

- Useful for chatbots, document question answering, etc.

- Context-aware for better response generation.

## 🚀 **Steps to Implement an Agent**

1. ✍️ **Prompt Template Creation**: Guide the agent on tasks and output formatting.

2. 🧩 **Output Parser Definition**: Convert LLM output into actionable responses.

3. 🧠 **LLM Setup**: Choose the preferred language model.

4. 🛑 **Stop Sequence Definition**: Instruct the LLM when to stop output.

5. 🤹 **Agent Setup**: Merge LLM, prompt template, output parser, and stop sequence.

6. 🏃 **Agent Usage**: Operate the agent using an executor with user inputs.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool 
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langsmith import Client
from langchain.agents import create_agent 



In [ ]:
# DuckDuckGo search
search = DuckDuckGoSearchRun()

search_tool = Tool(
    name="current_event_search",  # Tool names must match pattern: ^[a-zA-Z0-9_-]+$ (no spaces)
    description="Useful for when you need to search about current events.",
    func=search.run
)

# Wikipedia
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=None))

wiki_tool = Tool(
    name="noun_search",
    description="Useful for when you need to search for information about nouns.",
    func=wiki.run,
)

tools = [search_tool, wiki_tool]

In [58]:

client = Client(api_url=os.environ["LANGCHAIN_ENDPOINT"])

system_prompt = client.pull_prompt("moche/react")

In [59]:
print(system_prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [52]:
# View the template structure
print("Template variables:", system_prompt.input_variables)
print("\Template:")
print(system_prompt.template)

# For create_agent, you need to remove these placeholders:
# - {tools} - create_agent injects tool descriptions automatically
# - {tool_names} - create_agent handles this
# - {input} - create_agent uses messages format instead
# - {agent_scratchpad} - create_agent manages this internally

Template variables: ['agent_scratchpad', 'input', 'tool_names', 'tools']
\Template:
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


<>:3: SyntaxWarning: "\T" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\T"? A raw string is also an option.
<>:3: SyntaxWarning: "\T" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\T"? A raw string is also an option.
/var/folders/wj/6fzswsh9777b1q4rk7dhbbhr0000gn/T/ipykernel_41023/10452451.py:3: SyntaxWarning: "\T" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\T"? A raw string is also an option.
  print("\Template:")


In [53]:
llm = ChatOpenAI(model=open_ai_model, temperature=0.0)

In [60]:

# For create_agent, use a simple system prompt string
# The ReAct format is built into create_agent by default
agent = create_agent(
    model=llm, 
    tools=tools, 
    system_prompt="Answer the following questions as best you can. Use the available tools to help you."
)

In [61]:
agent.invoke({"messages": [{"role": "user", "content": "Who is playing in the Super Bowl in 2025? Where will it be played?"}]})

{'messages': [HumanMessage(content='Who is playing in the Super Bowl in 2025? Where will it be played?', additional_kwargs={}, response_metadata={}, id='db3b802a-17ac-4257-9a6d-b022fdd6f150'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1056, 'prompt_tokens': 208, 'total_tokens': 1264, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1024, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CeOCoJ8b9YIjxkfGjBjQHcPuBapPz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2d7ff1d4-2ae4-4388-acf3-456e579701ba-0', tool_calls=[{'name': 'current_event_search', 'args': {'__arg1': '2025 Super Bowl teams and location'}, 'id': 'call_RiWXbzB9A20dfMG3hMlIjFbS', 'type': 'tool_ca